In [1]:
!pip install psycopg2

In [2]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [3]:
 def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    return movie 

In [35]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_return(wiki_path,kaggle_path,ratings_path):
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    ratings = pd.read_csv(ratings_path)
    kaggle_metadata = pd.read_csv(kaggle_path)

    # Open and read the Wikipedia data JSON file.
    with open(wiki_path) as file:
        wiki_movies_raw = json.load(file) 
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw
    if ('Director' in movie or 'Directed by' in movie)
       and 'imdb_link' in movie
       and 'No. of episodes' not in movie]
    print (len(wiki_movies))

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies]

 
    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try: 
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})') 
        print(len(wiki_movies_df))
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        print(len(wiki_movies_df))
    except Exception as e:    # DEBUG exception
        print(e) 

        
        
    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
        
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ''.join(x) if type(x) == list else x)
    print(box_office)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\d+\.?\d*\s*[mb]illion'

    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

    # 12. Add the parse_dollars function.
    def parse_dollars():
         # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

    # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            
            print(s)

        # convert to float and multiply by a million
            value = float(s) * 10**6

         # return value
            return value

     # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

         # remove dollar sign and " billion"
             s = re.sub('\$|\s|[a-zA-Z]','', s)

         # convert to float and multiply by a billion
             value = float(s) * 10**9

         # return value
             return value

     # if input is of the form $###,###,###
     elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

         # remove dollar sign and commas
             s = re.sub('\$|,','', s)

         # convert to float
             value = float(s)

         # return value
             return value

     # otherwise, return NaN
     else:
             return np.nan
        parse_dollars()
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
        box_office.str.extract().parse_dollars()
    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
        budget.str.extract().parse_dollars()


    # 15. Clean the release date column in the wiki_movies_df DataFrame.
        budget.str.extract().parse_dollars()


    # 16. Clean the running time column in the wiki_movies_df DataFrame.
        running_time.str.extract().parse_dollars()

    
    # Return three variables. The first is the wiki_movies_df DataFrame
     return wiki_movies_df, kaggle_metadata, ratings     
#########     return wiki_movies_df, kaggle_metadata, ratings 
        


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 90)

In [31]:
for number in range(10):
    print(number)

0
1
2
3
4
5
6
7
8
9


In [32]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = "resources"
# The Wikipedia data
wiki_path = f'{file_dir}/wikipedia_movies.json'
# The Kaggle metadata
kaggle_path = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_path = f'{file_dir}/ratings.csv'

In [33]:
# 18. Set the three variables equal to the function created in D1.
wiki_movies_df, kaggle_df, ratings_df = extract_transform_return(wiki_path,kaggle_path,ratings_path)

C:\Users\Pikem\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  


7076
7076
7033
0          $21.4 million
1           $2.7 million
2            $57,718,089
3             $7,331,647
4       $6,939,946 (USA)
              ...       
7070       $19.4 million
7071       $41.9 million
7072       $76.1 million
7073       $38.4 million
7074        $5.5 million
Name: Box office, Length: 5485, dtype: object


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [ ]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

In [ ]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()